In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('Image/pepper.bmp')

def cvtGray(InputImage): # 3채널 이미지 input
    binaryImage = img[:,:,0]*0.2 + img[:,:,1]*0.5 + img[:,:,2]*0.3
    binaryImage = binaryImage.astype('uint8')
    return binaryImage

# grayScale Image
grey = cvtGray(img)
grey.shape

(512, 512)

In [2]:
def toHist(inputImage, bins):
    h, w = inputImage.shape[:2]
    hist = np.zeros(bins)
    width= 256 / bins
    inputImage = inputImage // width
    for i in range(h):
        for j in range(w):
            index = int(inputImage[i][j])
            hist[index] +=1 
            
    return hist

In [3]:
# histogram
hist = toHist(grey, 256)

In [4]:
# 히스토그램 값들의 합 === 전체 픽셀 수
sum(hist) == 512 * 512 

True

In [5]:
# Normalized histogram
hist_hat = hist / (512 * 512)

In [6]:
# calculate weight 
weight = np.zeros(256)
weight[0] = hist_hat[0] # 초깃값

for i in range(1,len(hist_hat)):
    weight[i] = weight[i-1] + hist_hat[i]

In [7]:
totalMean = np.sum(grey) / (512 * 512)
print( totalMean )

115.58695983886719


In [8]:
# calculate mean_0
mean_0 = np.zeros(256)

for i in range(1,256):
    mean_0[i] = (weight[i-1]*mean_0[i-1] + i * hist_hat[i]) / weight[i]

In [9]:
# calculate mean_1
mean_1 = np.zeros(256)
weight_1 = 1 - weight 

for i in range(256):
    if(1-weight[i]):
        mean_1[i] = ( totalMean - weight[i]*mean_0[i] ) / (1 - weight[i])
    else:
        pass

In [10]:
v_between = np.zeros(256)
for i in range(256):
    v_between[i] = weight[i]*(1-weight[i])*(mean_0[i] - mean_1[i])**2

treshold = np.argmax(v_between)
print(treshold)

115


In [11]:
grey[grey >= treshold] = 255

In [12]:
grey[grey < treshold] = 0

In [13]:
cv2.imshow('binary', grey)
cv2.waitKey(0)
cv2.destroyAllWindows()
grey

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0, 255,   0, ..., 255, 255, 255],
       ...,
       [  0, 255, 255, ..., 255, 255, 255],
       [  0, 255,   0, ..., 255, 255, 255],
       [  0,   0, 255, ..., 255, 255, 255]], dtype=uint8)